In [1]:
%matplotlib inline
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import stats
from ast import literal_eval # 문자열 모형의 딕트를 스근하게 딕트로 바꾸어 준다. 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

In [2]:
md = pd.read_csv('movieRecommaned/input/movies_metadata.csv')

FileNotFoundError: File b'movieRecommaned/input/movies_metadata.csv' does not exist

In [3]:
md.isnull().sum()

NameError: name 'md' is not defined

In [4]:
md.info()

NameError: name 'md' is not defined

In [5]:
md['genres'] = md['genres'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

NameError: name 'md' is not defined

In [6]:
md['year'] = md['release_date'].apply(lambda x:str(x).split('-')[0] if x != np.nan else np.nan)

NameError: name 'md' is not defined

In [7]:
vote_count = md['vote_count']
vote_average = md['vote_average']
m = vote_count.quantile(0.95)
c = vote_average.mean()
c

NameError: name 'md' is not defined

In [8]:
def get_wr(x):
    v = x['vote_count']
    r = x['vote_average']
    return (v / (v + m) * r) + (m/(v+m)*c)


In [9]:

qualified = md[(md['vote_count'].notnull()) & (md['vote_average'].notnull()) & (md['vote_count'] >= m)][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype(int)
qualified['vote_average'] = qualified['vote_average'].astype(int)

NameError: name 'md' is not defined

In [10]:
qualified.info()

NameError: name 'qualified' is not defined

In [11]:
qualified['wr'] = qualified.apply(get_wr, axis=1)

NameError: name 'qualified' is not defined

In [12]:
qualified = qualified.sort_values('wr', ascending=False)

NameError: name 'qualified' is not defined

In [13]:
s_gen = md.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)

NameError: name 'md' is not defined

In [14]:
s_gen.name = 'genres' 
s_gen = md.drop('genres', axis=1).join(s_gen)

NameError: name 's_gen' is not defined

In [15]:
def bar_chart(genres, percent=0.85):
    df = s_gen[s_gen['genres'] == genres]
    vote_count = df[df['vote_count'].notnull()]['vote_count']
    vote_average = df[df['vote_average'].notnull()]['vote_average']
    m = vote_count.quantile(percent)
    c = vote_average.mean()
    qualified = df[(df['vote_count'].notnull()) & (df['vote_count'] >= m) & df['vote_average'].notnull()][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype(int)
    qualified['vote_average'] = qualified['vote_average'].astype(int)
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * c), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    return qualified

In [16]:
bar_chart('Adventure').head(10)

NameError: name 's_gen' is not defined

In [17]:
bar_chart('Romance').head(3)

NameError: name 's_gen' is not defined

In [18]:
bar_chart('Thriller').head(3)

NameError: name 's_gen' is not defined

In [19]:
link_small = pd.read_csv('movieRecommaned/input/links_small.csv')
link_small = link_small[link_small['tmdbId'].notnull()]['tmdbId'].astype('int')

FileNotFoundError: File b'movieRecommaned/input/links_small.csv' does not exist

In [20]:
md = md.drop([19730, 29503, 35587])

NameError: name 'md' is not defined

In [21]:
md['id'] = md['id'].astype('int')

NameError: name 'md' is not defined

In [22]:
smd = md[md['id'].isin(link_small)]

NameError: name 'md' is not defined

In [23]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

NameError: name 'smd' is not defined

In [24]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

NameError: name 'smd' is not defined

In [25]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[0]

NameError: name 'tfidf_matrix' is not defined

In [26]:
smd = smd.reset_index()

NameError: name 'smd' is not defined

In [27]:
titles = smd['title']
indces = pd.Series(smd.index, index=titles)

NameError: name 'smd' is not defined

In [28]:
def getrecommandations(title):
    index = indces[title]
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores] 
    return titles.iloc[movie_indices]

In [29]:
getrecommandations('The Dark Knight')

NameError: name 'indces' is not defined

In [30]:
credits = pd.read_csv('movieRecommaned/input/credits.csv')
keywords = pd.read_csv('movieRecommaned/input/keywords.csv')

FileNotFoundError: File b'movieRecommaned/input/credits.csv' does not exist

In [31]:
keywords['id'] = keywords['id'].astype(int)
credits['id'] = credits['id'].astype(int)
md['id'] = md['id'].astype(int)

NameError: name 'keywords' is not defined

In [32]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

NameError: name 'md' is not defined

In [33]:
smd = md[md['id'].isin(link_small)]

NameError: name 'md' is not defined

In [34]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x :len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

NameError: name 'smd' is not defined

In [35]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [36]:
smd['director'] = smd['crew'].apply(get_director)

NameError: name 'smd' is not defined

In [37]:
smd['cast'] = smd['cast'].apply(lambda x : [i['name'] for i in x] if isinstance(x, list) else [])

NameError: name 'smd' is not defined

In [38]:
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) > 3 else x)

NameError: name 'smd' is not defined

In [39]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

NameError: name 'smd' is not defined

In [40]:
smd['cast'] = smd['cast'].apply(lambda x : [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype(str).apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x, x, x])

NameError: name 'smd' is not defined

In [41]:
s = smd.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)

NameError: name 'smd' is not defined

In [42]:
s = s.value_counts()
s[:5]

NameError: name 's' is not defined

In [43]:
s = s[s > 1]


NameError: name 's' is not defined

In [44]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [45]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [46]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)

NameError: name 'smd' is not defined

In [47]:
smd['keywords'] = smd['keywords'].apply(lambda x :[stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

NameError: name 'smd' is not defined

In [48]:
smd['soup'] = smd['keywords']+smd['cast']+smd['director']+smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

NameError: name 'smd' is not defined

In [49]:
count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

NameError: name 'smd' is not defined

In [50]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

NameError: name 'count_matrix' is not defined

In [51]:
smd = smd.reset_index()
titles = smd['title']
indces = pd.Series(smd.index, index=smd['title'])

NameError: name 'smd' is not defined

In [52]:
indces

NameError: name 'indces' is not defined

In [53]:
getrecommandations('The Dark Knight')

NameError: name 'indces' is not defined

In [54]:
def improved_recommendations(title):
    index = indces[title]
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indecs = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indecs][['title', 'vote_count', 'vote_average', 'year']]
    vote_count = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_average = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    c = vote_average.mean()
    m = vote_count.quantile(0.6)
    qualified = movies[(movies['vote_count'].notnull()) & (movies['vote_average'].notnull()) & (movies['vote_count']>m)]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(get_wr, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [55]:
reader = Reader()

In [56]:
raitings = pd.read_csv('movieRecommaned/input/ratings_small.csv')
raitings.head()

FileNotFoundError: File b'movieRecommaned/input/ratings_small.csv' does not exist

In [57]:
data = Dataset.load_from_df(raitings[['userId', 'movieId', 'rating']], reader=reader)
data.split(n_folds=5)

NameError: name 'raitings' is not defined

In [58]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

NameError: name 'data' is not defined

In [59]:
trainset = data.build_full_trainset()
svd.train(trainset)

NameError: name 'data' is not defined

In [60]:
raitings[raitings['userId']==1]

NameError: name 'raitings' is not defined

In [61]:
svd.predict(1, 302)

AttributeError: 'SVD' object has no attribute 'trainset'

In [62]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [63]:
id_map = pd.read_csv('movieRecommaned/input/links_small.csv')[['movieId','tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']

FileNotFoundError: File b'movieRecommaned/input/links_small.csv' does not exist

In [64]:
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

NameError: name 'id_map' is not defined

In [65]:
indices_map = id_map.set_index('id')

NameError: name 'id_map' is not defined

In [66]:
def hybrid(userId, title):
    indces[title]

In [67]:
indces

NameError: name 'indces' is not defined